In [ ]:
!pip install crewai crewai-tools

In [ ]:
import os
from typing import Dict, Type
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [8]:
department_loads = {
    'payments_and_billing': 0,
    'shipping_and_logistics': 0,
    'warehouse': 0,
    'manufacturing_and_quality': 0,
    'product_quality': 0,
    'general_and_feedback': 0,
}

class GetDepartmentLoadsInput(BaseModel):
    pass

class UpdateDepartmentLoadInput(BaseModel):
    department: str = Field(..., description="Name of the department to update load for")


class GetDepartmentLoadsTool(BaseTool):
    name: str = "get_department_loads"
    description: str = "Get the current workload for all departments"
    args_schema: Type[BaseModel] = GetDepartmentLoadsInput

    def _run(self) -> Dict[str, int]:
        return department_loads

class UpdateDepartmentLoadTool(BaseTool):
    name: str = "update_department_load"
    description: str = "Increment the workload counter for a specified department"
    args_schema: Type[BaseModel] = UpdateDepartmentLoadInput

    def _run(self, department: str) -> Dict[str, int]:
        if department in department_loads:
            department_loads[department] += 1
        return department_loads


In [9]:
def create_customer_support_crew(openai_api_key: str) -> Crew:

    supervisor = Agent(
        role='Supervisor',
        goal='Ensure effective ticket resolution by reviewing key aspects of classification, routing and response',
        backstory="""Senior support manager who reviews tickets for critical issues only.
        Only disapprove if there are significant problems:

        Classification issues:
        - Wrong category entirely
        - Urgency completely mismatched (e.g., ignoring explicit emergency)
        - Missing critical details

        Routing issues:
        - Wrong department
        - Ignoring extreme department overload (>2x normal load)

        Response issues:
        - Wrong/missing department name
        - Wrong timeline
        - Rude or dismissive tone

        Minor style preferences or suggestions should be noted but should not trigger disapproval.""",
        allow_delegation=True,
        verbose=True,
        max_iter=2,
        llm_config={'api_key': openai_api_key}
    )

    classifier = Agent(
        role='Ticket Classifier',
        goal='Accurately classify support tickets',
        backstory="""Specialist in customer support ticket classification.

        Categories:
        - Shipping Issues: Missing packages, delivery problems, tracking issues
        - Payments/Billing: Payment processing, refunds, disputes
        - Warehouse: Inventory, stock
        - Manufacturing: Product defects, quality
        - General: Inquiries, feedback

        HIGH Priority Triggers:
        - Explicit urgency words: 'urgent', 'emergency', 'asap'
        - Implicit urgency: 'need help', 'need assistance', 'worried'
        - Time-sensitive issues: gifts, events, deadlines
        - Security concerns: theft, fraud, missing items >$200

        MEDIUM Priority: Standard delays, product issues
        LOW Priority: General inquiries, feedback""",
        verbose=True,
        max_iter=2,
        llm_config={'api_key': openai_api_key}
    )

    router = Agent(
        role='Ticket Router',
        goal='Route tickets to appropriate departments based on specialization and load',
        backstory="""Expert in routing tickets to appropritate departments within the organization based on the content of the ticket to ensure customer issues are remedied by the
                     right department within the organization. Also takes each departments' load into consideration before routing tickets. In case, a particular department is
                     experiencing heavy load, the tickets must be directed to 'General and Feedback' department.""",
        verbose=True,
        llm_config={'api_key': openai_api_key},
        max_iter=2,
        tools=[GetDepartmentLoadsTool(), UpdateDepartmentLoadTool()]
    )

    responder = Agent(
        role='Response Composer',
        goal='Craft professional yet friendly, empathetic responses that reassure customers',
        backstory="""You are a customer service AI tasked with creating professional and friendly responses to customer inquiries. Your goal is to acknowledge receipt of the issue, identify the appropriate handling department, and provide a timeline for resolution.

                    Here are the departments you can assign issues to:
                    - Shipping and Logistics
                    - Payments and Billing
                    - Warehouse
                    - Manufacturing and Quality
                    - General and Feedback""",
        verbose=True,
        max_iter=2,
        llm_config={'api_key': openai_api_key},
    )

    classification_task = Task(
        description="""Your task is to analyze customer support tickets and classify them into the appropriate category and priority level.  \
                      Here's how you should approach this task:

                      First, carefully read the following ticket text:

                      Now, consider the following categories and their descriptions:

                      1. Shipping and Logistics:
                        - Missing packages
                        - Delivery problems
                        - Tracking issues
                        - Any concerns related to the shipment of products

                      2. Payments/Billing:
                        - Payment processing problems
                        - Refund requests or issues
                        - Billing disputes
                        - Any financial transactions or concerns

                      3. Warehouse:
                        - Inventory problems
                        - Stock availability issues
                        - Order fulfillment concerns

                      4. Manufacturing:
                        - Product defects
                        - Quality control issues
                        - Problems related to the production of items

                      5. General:
                        - General inquiries
                        - Feedback
                        - Any issues that don't clearly fit into the above categories

                      Next, consider the priority levels and their triggers:

                      HIGH Priority:
                      - Explicit urgency words: 'urgent', 'emergency', 'asap', 'immediately'
                      - Implicit urgency phrases: 'need help', 'need assistance', 'worried', 'concerned'
                      - Time-sensitive issues: gifts, events, deadlines, perishable items
                      - Security concerns: theft, fraud, missing items valued at over $200
                      - Legal threats or mentions of legal action
                      - Mentions of harm, danger, or safety issues

                      MEDIUM Priority:
                      - Standard shipping delays
                      - Common product issues or defects
                      - Billing discrepancies under $200
                      - Requests for refunds or exchanges
                      - Complaints about product quality or functionality

                      LOW Priority:
                      - General inquiries about products or services
                      - Feedback or suggestions not requiring immediate action
                      - Updates to customer information
                      - Newsletter subscriptions or unsubscriptions

                      To classify the ticket:

                      1. Carefully analyze the ticket text for keywords, phrases, and context that align with the categories and priority levels described above.

                      2. Determine the most appropriate category based on the main issue presented in the ticket. If multiple issues are present, choose the most prominent or urgent one.

                      3. Assess the priority level by looking for triggers mentioned in the priority descriptions. Consider both explicit and implicit indicators of urgency.

                      4. If the ticket contains elements that could fit multiple categories or priority levels, explain your reasoning for choosing one over the others.

                      Provide your classification in the following format:

                      Category: [Insert chosen category]
                      Priority: [Insert priority level]
                      Reasoning:[Provide a detailed explanation of why you chose this category and priority level. Include specific phrases or keywords from the ticket text that support your decision. Discuss any challenging aspects of the classification, if applicable.]

                      Remember to be thorough in your analysis and clear in your explanations. Your goal is to ensure accurate and consistent classification of customer support tickets.

                      """,
        agent=classifier,
        expected_output="""Dict with three fields: {{"category": "chosen_category", "priority": "priority_level", "key_info": "extracted_info"}}"""
    )

    review_classification_task = Task(
        description="""Review the ticket classification for:
                      1. Category accuracy among issues related to Shipping and Logistics, Payments and Billing, Warehouse, Manufacturing and Quality, General and Feedback
                      2. Priority level appropriateness among LOW, MEDIUM, HIGH
                      3. Key information extraction completeness
                      Provide specific feedback if improvements needed.""",
        agent=supervisor,
        context=[classification_task],
        expected_output="""Dict with two fields: {{"approved_status": "approved" or "not_approved", "feedback": "feedback"}}"""
    )

    routing_task = Task(
        description="""Determine optimal department routing considering:
                      1. Ticket category and priority
                      2. Current department workloads
                      3. Department specializations where the departments are Shipping and Logistics (missing package, never received, tracking, delayed shipping, delivered in poor condition),
                      Payments and Billing (returns, refunds, payment not reflecting, unable to pay), Warehouse (availability of products, inventory, sold out, restock), Manufacturing and Quality
                      (product is not up to mark), General and Feedback (anything related to feedback, suggestions and improvements that can be made)
                      4. Load balancing requirements (route to appropriate department, but if said department is under heavy load, as in, double the load of 'General and Feedback'
                      then route to General and Feedback)""",
        agent=router,
        expected_output="Dict with two fields: routing decision and department load update"
    )

    review_routing_task = Task(
        description="""Review the routing decision for:
                      1. Department appropriateness where the departments are Shipping and Logistics, Payments and Billing, Warehouse, Manufacturing and Quality, General and Feedback
                      2. Load balancing effectiveness where the ticket is routed to a different department if the respective department is experiencing heavy load (heavy load is when a department is
                      dealing with over 5x the number of tickets compared to other departments)
                      3. Priority consideration
                      Provide specific feedback if improvements needed.""",
        agent=supervisor,
        context=[routing_task],
        expected_output="""Dict with two fields: {{"approved_status": "approved" or "not_approved", "feedback": "feedback"}}"""
    )

    response_task = Task(
        description="""
                      To determine the handling department, analyze the content of the ISSUE. Look for keywords or themes that align with one of the departments listed above. If the issue doesn't clearly fit into a specific department, assign it to "General and Feedback".

                      Set the timeline for resolution based on the current LOAD:
                      - If LOAD is "normal", use "within 7 days"
                      - If LOAD is "high", use "within 10 days"

                      Use the following template to craft your response:
                      "Hi, thanks for letting us know about [issue]. Our [department] team is looking into this and will get back to you [timeline]."

                      To create your final response:
                      1. Start with "Hi, thanks for letting us know about "
                      2. Briefly summarize the ISSUE in a few words
                      3. Add ". Our "
                      4. Insert the name of the handling department you determined
                      5. Add " team is looking into this and will get back to you "
                      6. Insert the appropriate timeline based on the LOAD
                      7. End with a period

                      Ensure your response is professional yet friendly in tone. Avoid using technical jargon or overly formal language.

                      Write your final response inside <response> tags.

                      Here are two examples to guide you:

                      Example 1:
                      ISSUE = "My package hasn't arrived yet and it's been 2 weeks"
                      LOAD = "normal"

                      <response>
                      Hi, thanks for letting us know about your delayed package. Our Shipping and Logistics team is looking into this and will get back to you within 7 days.
                      </response>

                      Example 2:
                      ISSUE = "The product I received has a manufacturing defect"
                      LOAD = "high"

                      <response>
                      Hi, thanks for letting us know about the manufacturing defect. Our Manufacturing and Quality team is looking into this and will get back to you within 10 days.
                      </response>

                      Now, create a response based on the provided ISSUE and LOAD.
        """,
        agent=responder,
        expected_output="Response message as string"
    )
    print(f"Response to customer: {response_task.output}")

    review_response_task = Task(
        description="""Review the response for:
                          1. Correct department name used
                          2. Correct timeline (7 or 10 days) based on load
                          3. Follows exact template format as follows:

                          ISSUE = "My package hasn't arrived yet and it's been 2 weeks"
                          LOAD = "normal"

                          <response>
                          Hi, thanks for letting us know about your delayed package. Our Shipping and Logistics team is looking into this and will get back to you within 7 days.
                          </response>

                          Example 2:
                          ISSUE = "The product I received has a manufacturing defect"
                          LOAD = "high"

                          <response>
                          Hi, thanks for letting us know about the manufacturing defect. Our Manufacturing and Quality team is looking into this and will get back to you within 10 days.
                          </response>

                          Minor deviations from the template does not warrant disapproval, as long as the message is conveyed, the objective is met.""",
        agent=supervisor,
        context=[response_task],
        expected_output="""Dict with two fields: {{"approved_status": "approved" or "not_approved", "feedback": "feedback"}}"""
    )

    crew = Crew(
        agents=[supervisor, classifier, router, responder],
        tasks=[
            classification_task,
            review_classification_task,
            routing_task,
            review_routing_task,
            response_task,
            review_response_task
        ],
        verbose=True
    )

    return crew


In [10]:
if __name__ == "__main__":
    ticket = """
        'id': 'TKT-2025-0121-4832',
        'content': '''My order (#ORDER-58921) was marked as delivered yesterday, but I never
        received it. I checked my front porch and mailbox multiple times, and checked with
        my neighbors, but the package is missing. The tracking shows it was
        delivered at 2:30 PM, but I was home all day and didn't hear any delivery.
        This was a $299 limited edition collector's item that's now sold out, and I'm really
        worried it might have been stolen. I need help locating my package urgently as this
        was a birthday gift for my son this weekend. Order details:
        - Shipping address: 1234 Pine Street, Portland, OR 97201
        - Carrier: FedEx
        - Tracking number: FX857492634US''',
        'customer_id': 'CUST-89321',
        'timestamp': '2025-01-21T09:15:23Z',
        'order_number': 'ORDER-58921',
        'channel': 'email'"""

    crew = create_customer_support_crew(openai_api_key='your-api-key')
    result = crew.kickoff(inputs={'ticket': ticket})
    print(f"Processed ticket result: {result}")

LLM value is None
LLM value is None
LLM value is None
LLM value is None
Response to customer: None
# Agent: Ticket Classifier
## Task: Your task is to analyze customer support tickets and classify them into the appropriate category and priority level.                        Here's how you should approach this task:

                      First, carefully read the following ticket text:

                      Now, consider the following categories and their descriptions:

                      1. Shipping and Logistics:
                        - Missing packages
                        - Delivery problems
                        - Tracking issues
                        - Any concerns related to the shipment of products

                      2. Payments/Billing:
                        - Payment processing problems
                        - Refund requests or issues
                        - Billing disputes
                        - Any financial transactions or concerns

            

In [12]:
    tickets = [
        {
            'id': 'TKT-2025-0123-6104',
            'content': '''I recently purchased a Smart Home Security Camera (Model SHC-2000)
            and it's not functioning correctly. The camera connects to WiFi but doesn't
            record video or send motion alerts as advertised. I've already tried:
            - Resetting the device
            - Updating firmware
            - Changing WiFi networks
            None of these steps resolved the issue. This is a $249 device, and I expect it
            to work as described in the product manual. I need a replacement or full refund.
            - Purchase date: 2025-01-15
            - Serial number: SN-SHC2000-1247''',
            'customer_id': 'CUST-54321',
            'timestamp': '2025-01-23T14:22:11Z',
            'order_number': 'ORDER-59562',
            'channel': 'phone'
        },
        {
            'id': 'TKT-2025-0124-7236',
            'content': '''I noticed an unauthorized charge on my account for $129.99
            from your company. I did not place this order, and it appears on my statement
            dated 2025-01-20. I immediately want to:
            1. Dispute this charge
            2. Ensure my account is secure
            3. Get a refund processed
            The transaction details:
            - Transaction ID: TX-89654
            - Last 4 digits of card: 7412
            - Transaction time: 2025-01-20T22:15:33Z''',
            'customer_id': 'CUST-23456',
            'timestamp': '2025-01-24T09:45:22Z',
            'order_number': 'ORDER-595341',
            'channel': 'email'
        },
        {
            'id': 'TKT-2025-0125-8147',
            'content': '''I've been trying to cancel my annual subscription to your
            software service for over a week. The online cancellation process is not
            working, and I've been charged again for another year. I want:
            - Immediate cancellation of my subscription
            - Prorated refund for the remaining months
            - Confirmation of cancellation in writing
            Subscription details:
            - Subscription ID: SUB-2024-76543
            - Last renewal: 2025-01-01
            - Billing cycle: Annual
            - Amount charged: $199.99''',
            'customer_id': 'CUST-87654',
            'timestamp': '2025-01-25T11:33:54Z',
            'order_number': 'ORDER-59860',
            'channel': 'support_portal'
        }
    ]

    # for ticket in tickets:
        # result = crew.kickoff(inputs={'ticket': ticket})
        # print(f"Processed ticket result: {result}")